In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import skimage as sk
import skimage.io as io
import matplotlib
%matplotlib inline
from skimage.util import img_as_ubyte
import utils, morph
from main import *

NameError: name 'Delaunay' is not defined

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import interp2d

In [ ]:
from pathlib import Path
data = Path("input")
data.mkdir(parents=True, exist_ok=True)

In [ ]:
# matplotlib.use("TkAgg")
# matplotlib.interactive(True)

In [ ]:
im1 = utils.setup_img('me')

In [69]:
im1.shape, im1.dtype

((575, 547, 3), dtype('float64'))

In [70]:
im2.shape, im2.dtype

((575, 547, 3), dtype('float64'))

In [80]:
avg_pts = avg_points(im1_pts, im2_pts, 0.8)

In [83]:
delaunay_tri = Delaunay(avg_pts)
type(delaunay_tri)

scipy.spatial.qhull.Delaunay

In [79]:
import matplotlib.pyplot as plt
plot_tri_mesh(im1, avg_pts, delaunay_tri)
plt.show()

In [51]:
plt.imshow(im1)
tri.plot_tri_mesh(im1_pts, delaunay_tri)

In [50]:
plt.imshow(im2)
tri.plot_tri_mesh(im2_pts, delaunay_tri)